# Discrete Kalman Filter

In [1]:
import numpy as np
import pandas as pd
from scipy.linalg import expm

### Helper Functions

In [2]:
def predictStep(x, A):
    xPred = np.matmul(A,x)
    return xPred

In [3]:
def innovationStep(xPred,z, H):
    innovation = np.subtract(z, np.matmul(H, xPred)) #innovation
    return innovation

In [4]:
def UpdateStep(xPred, nu, K):
    xNew = np.add(xPred, np.matmul(K, innovation))
    return xNew

In [542]:
def dataAssociation(centroidFrame, filteredCentroidPolarFrames):
    rThetaMeasurement = convertCartesianToPolar(centroidFrame)
    #predict
    previousCentroids = filteredCentroidPolarFrames[-1]
    xPredAll = np.array([])
    for centroidIndex in list(previousCentroids['CentroidNumber']):
        xPrev = np.expand_dims(previousCentroids.loc[centroidIndex][:-1].values, axis=1)
        xPred = predictStep(xPrev, A)
        if len(xPredAll) == 0:
            xPredAll = xPred
        else:
            xPredAll = np.hstack((xPredAll, xPred))

    xPredAll = np.transpose(xPredAll) #keep consistent with other dataframes
    xPredDf = pd.DataFrame(xPredAll)
    xPredDf.columns = ['PredictedRange', 'PredictedDoppler', 'PredictedTheta', 'PredictedAngularVelocity']
    xPredDf['CentroidNumber'] = pd.Series(np.arange(xPredDf.shape[0]))

    #data associate
    distanceMatrix = np.eye(xPredDf.shape[0], rThetaMeasurement.shape[0])
    #create distance matrix
    for centroidIndex in xPredDf['CentroidNumber']:
        #access prev centroid information
        prevCentroidInfo = xPredDf.loc[centroidIndex]
        #compare against measurement centroid stats and find distances
        for centroidNumber in rThetaMeasurement['CentroidNumber']:
            #access measured centroid information
            measurementCentroidInfo = rThetaMeasurement.loc[centroidNumber]
            #find distances between measured and predicted centroids
            length = np.sqrt((prevCentroidInfo.PredictedRange**2) + (measurementCentroidInfo.R ** 2) - \
            (2*measurementCentroidInfo.R*prevCentroidInfo.PredictedRange*np.cos((prevCentroidInfo.PredictedTheta - measurementCentroidInfo.Theta))))
            #fill in distance matrix
            distanceMatrix[centroidIndex, centroidNumber] = length
    #global association
    correctAssociationMatrix = np.array([])
    #loop as many times as needed
    for loopIterator in range(0, min(max(xPredDf['CentroidNumber']),max(rThetaMeasurement['CentroidNumber']))+1) :
        #find minimum distance 
        predictedCentroidNumber, measuredCentroidNumber = (np.where(distanceMatrix == np.min(distanceMatrix))[0][0], \
                                                           np.where(distanceMatrix == np.min(distanceMatrix))[1][0])
        if len(correctAssociationMatrix) == 0:
            correctAssociationMatrix = np.array([rThetaMeasurement.loc[measuredCentroidNumber].R,\
                                                        rThetaMeasurement.loc[measuredCentroidNumber].Theta,predictedCentroidNumber])
        else:
            correctAssociationMatrix = np.vstack((correctAssociationMatrix,\
                                                  np.array([rThetaMeasurement.loc[measuredCentroidNumber].R,\
                                                            rThetaMeasurement.loc[measuredCentroidNumber].Theta,predictedCentroidNumber])))

        #fill in with Inf where the centroid has been associated
        distanceMatrix[predictedCentroidNumber, :] = np.Inf

    associatedDf = pd.DataFrame(correctAssociationMatrix, columns=['MeasuredRange', 'MeasuredTheta', 'CentroidNumber'])
    print('DONE ASSOCIATING')

    #measurement has more centroids than associated
    if associatedDf.shape[0] < rThetaMeasurement.shape[0]:
        print('MORE')
        #add those extra centoids 
        for centroidIndex in range(0,rThetaMeasurement.shape[0]):
            measurement = rThetaMeasurement.loc[centroidIndex]
            if measurement.R not in list(associatedDf['MeasuredRange']):
                measurement = {'MeasuredRange': measurement.R, 'MeasuredTheta':measurement.Theta, 'CentroidNumber': max(associatedDf['CentroidNumber']+1)}
                measurement = pd.DataFrame(data=measurement, index=range(1))
                associatedDf = pd.concat([associatedDf,measurement], axis=0, ignore_index = True)                       

    #measurement has less centroids than predicted - add the predicted value of the unassociated centroid into the associatedDf
    if not((np.isinf(distanceMatrix)).all()):
        print('LESS')
        #add predicted value of unassociated centroid to the dataframe
        np.isfinite(distanceMatrix[:,0])
        for centroidIndex in range(0,xPredDf.shape[0]):
            predicted = xPredDf.loc[centroidIndex]
            if predicted.CentroidNumber not in list(associatedDf['CentroidNumber']):
                predicted = {'MeasuredRange': predicted.Range, 'MeasuredTheta':predicted.Theta, 'CentroidNumber': predicted.CentroidNumber}
                predicted = pd.DataFrame(data=predicted, index=range(1))
                associatedDf = pd.concat([associatedDf,predicted], axis=0, ignore_index = True)    
    return associatedDf, xPredDf

In [6]:
#convert x,y to r,theta
def convertCartesianToPolar(cartesianMeasurementCentroidDf):
    measurementR = np.sqrt(np.square(cartesianMeasurementCentroidDf['X']) + np.square(cartesianMeasurementCentroidDf['Y']))
    measurementTheta = np.arctan(cartesianMeasurementCentroidDf['Y']/cartesianMeasurementCentroidDf['X'])
    polarCentroidDf = pd.DataFrame({'CentroidNumber':cartesianMeasurementCentroidDf['CentroidNumber'], 
                                   'R': measurementR,
                                   'Theta': measurementTheta})
    return polarCentroidDf

In [7]:
#initialize variables
deltaT = 50*10**-3 #50ms

#system matrix
A = np.array([
    [1, deltaT, 0,0],
    [0,1,0,0],
    [0,0,1,deltaT],
    [0,0,0,1]
])
#state transition matrix
F = expm(A*deltaT)
#output matrix
H = np.array([[1,0,0,0],
              [0,0,1,0]])
#covariance matrices
Q = np.eye(4)
R = np.ones(2).reshape(-1,1)
Pc = np.eye(4)

Pd = np.add(np.matmul(np.matmul(A,Pc), np.transpose(A)), Q) #prediction covariance
S = np.add(R, np.matmul(np.matmul(H, Pd), np.transpose(H))) #innovation covariance
K = np.matmul(Pd, np.matmul(np.transpose(H), np.linalg.inv(S))) #kalman gain
Pupdate = np.subtract(Pd, np.matmul(np.matmul(K, S), np.transpose(K))) #update covariance

### Unit Testing

In [8]:
#test predict step
x = np.expand_dims(np.array([1,0.5,0,0]), axis=1) #previous state
xPred = predictStep(x, A) #predicted state
xPred

array([[1.025],
       [0.5  ],
       [0.   ],
       [0.   ]])

In [9]:
#test innovation
#use xPred from previous step
# innovation(xPred, Pd, z, H, R)
z = np.expand_dims(np.array([1,0.5]), axis=1) #create column vector
innovation = innovationStep(xPred, z, H)
innovation

array([[-0.025],
       [ 0.5  ]])

In [10]:
#test update step
xNew = UpdateStep(xPred, innovation, K)
xNew

array([[0.88132417],
       [0.49641259],
       [0.38132417],
       [0.0095212 ]])

In [555]:
#test centroid dataset
#centroidDf_A contains 2 centroids
centroidDfCartesian_A = pd.DataFrame({'CentroidNumber':np.arange(0,2), 
                             'X':np.array([-1,0]), 
                             'Y':np.array([3,1])})
#centroidDf_B contains 2 centroids and continues on from centroidDf_A
centroidDfCartesian_B = pd.DataFrame({'CentroidNumber':np.arange(0,2), 
                             'X':np.array([-1,0.5]), 
                             'Y':np.array([2.5,0.7])})
#centroidDf_C contains 3 centroids continuing from centroidDf_B
centroidDfCartesian_C = pd.DataFrame({'CentroidNumber':np.arange(0,3), 
                             'X':np.array([-1,0.7, 0.3]), 
                             'Y':np.array([1.2,0.5, 3.2])})

#contains the centroid frames, where each frame represents a mmWave capture frame
# centroidFramesCartesianMeasurement = [centroidDfCartesian_A,centroidDfCartesian_B,centroidDfCartesian_C]
centroidFramesCartesianMeasurement = [centroidDfCartesian_A, centroidDfCartesian_B,centroidDfCartesian_C]

In [556]:
filteredCentroidPolarFrames = list()

#load in frames
for centroidFrame in centroidFramesCartesianMeasurement:
    filteredPolarFrame = np.array([])
    
    if len(filteredCentroidPolarFrames) == 0: #first iteration of the kalman filter
        polarCentroidFrameMeasurement = convertCartesianToPolar(centroidFrame)
        for centroid in polarCentroidFrameMeasurement['CentroidNumber']:
            xPred = np.expand_dims(np.array([0,0,0,0]), axis=1) #initialise 
            xPred = predictStep(x, A)
            measurement = np.expand_dims(polarCentroidFrameMeasurement.values[centroid][1:], axis=1)
            innovation = innovationStep(xPred, measurement, H)
            xUpdate = UpdateStep(xPred, innovation, K)
            if len(filteredPolarFrame) == 0:
                filteredPolarFrame = xUpdate
            else:
                filteredPolarFrame = np.hstack((filteredPolarFrame,xUpdate))
    else:
        
        associatedDf, xPredDf = dataAssociation(centroidFrame, filteredCentroidPolarFrames)
        overallDf = pd.merge(associatedDf, xPredDf, on='CentroidNumber', how='inner')
        for index in range(0 , overallDf.shape[0]):
            centroidData = overallDf.loc[index]
            measurement = np.expand_dims(np.array([centroidData.MeasuredRange, centroidData.MeasuredTheta]), axis=1)
            xPred = np.expand_dims(np.array([centroidData.PredictedRange, centroidData.PredictedDoppler,centroidData.PredictedTheta, centroidData.PredictedAngularVelocity]), axis=1)
            innovation = innovationStep(xPred, measurement, H)
            xUpdate = UpdateStep(xPred, innovation, K)
            if len(filteredPolarFrame) == 0:
                filteredPolarFrame = xUpdate
            else:
                filteredPolarFrame = np.hstack((filteredPolarFrame,xUpdate))
                
    filteredPolarDf = pd.DataFrame(np.transpose(filteredPolarFrame))
    filteredPolarDf.columns = ['Filtered Range', 'Filtered Doppler', 'Filtered Theta', 'Filtered Angular Velocity']
    filteredPolarDf['CentroidNumber'] = pd.Series(np.arange(filteredPolarDf.shape[0]))
    filteredCentroidPolarFrames.append(filteredPolarDf)

DONE ASSOCIATING
DONE ASSOCIATING
MORE


KeyError: 'Range'

In [552]:
centroidData = overallDf.loc[0]
measurement = np.expand_dims(np.array([centroidData.MeasuredRange, centroidData.MeasuredTheta]), axis=1)
xPred = np.expand_dims(np.array([centroidData.PredictedRange, centroidData.PredictedDoppler,centroidData.PredictedTheta, centroidData.PredictedAngularVelocity]), axis=1)


[[0.86023253]
 [0.95054684]]


In [550]:
centroidData

MeasuredRange               0.860233
MeasuredTheta               0.950547
CentroidNumber              1.000000
PredictedRange              0.638279
PredictedDoppler            0.489733
PredictedTheta              1.186068
PredictedAngularVelocity    0.029578
Name: 0, dtype: float64

In [539]:
rThetaMeasurement = convertCartesianToPolar(centroidFrame)
#predict
previousCentroids = filteredCentroidPolarFrames[-1]
xPredAll = np.array([])
for centroidIndex in list(previousCentroids['CentroidNumber']):
    xPrev = np.expand_dims(previousCentroids.loc[centroidIndex][:-1].values, axis=1)
    xPred = predictStep(xPrev, A)
    if len(xPredAll) == 0:
        xPredAll = xPred
    else:
        xPredAll = np.hstack((xPredAll, xPred))

xPredAll = np.transpose(xPredAll) #keep consistent with other dataframes
xPredDf = pd.DataFrame(xPredAll)
xPredDf.columns = ['Predicted_Range', 'Predicted_ Doppler', 'Predicted_Theta', 'Predicted_Angular_Velocity']
xPredDf['CentroidNumber'] = pd.Series(np.arange(xPredDf.shape[0]))

#data associate
distanceMatrix = np.eye(xPredDf.shape[0], rThetaMeasurement.shape[0])
#create distance matrix
for centroidIndex in xPredDf['CentroidNumber']:
    #access prev centroid information
    prevCentroidInfo = xPredDf.loc[centroidIndex]
    #compare against measurement centroid stats and find distances
    for centroidNumber in rThetaMeasurement['CentroidNumber']:
        #access measured centroid information
        measurementCentroidInfo = rThetaMeasurement.loc[centroidNumber]
        #find distances between measured and predicted centroids
        length = np.sqrt((prevCentroidInfo."Predicted Range"**2) + (measurementCentroidInfo.R ** 2) - \
        (2*measurementCentroidInfo.R*prevCentroidInfo.Range*np.cos((prevCentroidInfo.Theta - measurementCentroidInfo.Theta))))
        #fill in distance matrix
        distanceMatrix[centroidIndex, centroidNumber] = length
#global association
correctAssociationMatrix = np.array([])
#loop as many times as needed
for loopIterator in range(0, min(max(xPredDf['CentroidNumber']),max(rThetaMeasurement['CentroidNumber']))+1) :
    #find minimum distance 
    predictedCentroidNumber, measuredCentroidNumber = (np.where(distanceMatrix == np.min(distanceMatrix))[0][0], \
                                                       np.where(distanceMatrix == np.min(distanceMatrix))[1][0])
    if len(correctAssociationMatrix) == 0:
        correctAssociationMatrix = np.array([rThetaMeasurement.loc[measuredCentroidNumber].R,\
                                                    rThetaMeasurement.loc[measuredCentroidNumber].Theta,predictedCentroidNumber])
    else:
        correctAssociationMatrix = np.vstack((correctAssociationMatrix,\
                                              np.array([rThetaMeasurement.loc[measuredCentroidNumber].R,\
                                                        rThetaMeasurement.loc[measuredCentroidNumber].Theta,predictedCentroidNumber])))

    #fill in with Inf where the centroid has been associated
    distanceMatrix[predictedCentroidNumber, :] = np.Inf

associatedDf = pd.DataFrame(correctAssociationMatrix, columns=['Measured Range', 'Measured Theta', 'CentroidNumber'])
print('DONE ASSOCIATING')

#measurement has more centroids than associated
if associatedDf.shape[0] < rThetaMeasurement.shape[0]:
    print('MORE')
    #add those extra centoids 
    for centroidIndex in range(0,rThetaMeasurement.shape[0]):
        measurement = rThetaMeasurement.loc[centroidIndex]
        if measurement.R not in list(associatedDf['Range']):
            measurement = {'Measured Range': measurement.R, 'Measured Theta':measurement.Theta, 'CentroidNumber': max(associatedDf['CentroidNumber']+1)}
            measurement = pd.DataFrame(data=measurement, index=range(1))
            associatedDf = pd.concat([associatedDf,measurement], axis=0, ignore_index = True)                       

#measurement has less centroids than predicted - add the predicted value of the unassociated centroid into the associatedDf
if not((np.isinf(distanceMatrix)).all()):
    print('LESS')
    #add predicted value of unassociated centroid to the dataframe
    np.isfinite(distanceMatrix[:,0])
    for centroidIndex in range(0,xPredDf.shape[0]):
        predicted = xPredDf.loc[centroidIndex]
        if predicted.CentroidNumber not in list(associatedDf['CentroidNumber']):
            predicted = {'Measured Range': predicted.Range, 'Measured Theta':predicted.Theta, 'CentroidNumber': predicted.CentroidNumber}
            predicted = pd.DataFrame(data=predicted, index=range(1))
            associatedDf = pd.concat([associatedDf,predicted], axis=0, ignore_index = True)    

SyntaxError: invalid syntax (<ipython-input-539-cbc1eba16218>, line 29)

In [486]:
distanceMAtrixTest = distanceMatrix[2,:] = np.inf

In [381]:
measurement

array([3.21403174, 1.47731955, 2.        ])

In [335]:
rThetaMeasurement['CentroidNumber']

0    0
1    1
2    2
Name: CentroidNumber, dtype: int32

In [170]:
#unit test - DATA ASSOCIATION
rThetaMeasurement = convertCartesianToPolar(centroidDfCartesian_B) #next measurement 

previousCentroids = filteredCentroidPolar[-1]
xPredAll = np.array([])
#predict
for centroidIndex in list(previousCentroids.columns):
    xPrev = np.expand_dims(previousCentroids[centroidIndex].values, axis=1)
    xPred = predictStep(xPrev, A)
    if len(xPredAll) == 0:
        xPredAll = xPred
    else:
        xPredAll = np.vstack((xPredAll, xPred))
        
xPredDf = pd.DataFrame(xPredAll)
xPredDf.index = ['Range', 'Doppler', 'Theta', 'Angular Velocity']

#data associate and update
distanceMatrix = np.eye(xPredDf.shape[1], rThetaMeasurement.shape[0])
for centroidIndex in xPredDf.columns:
    #access prev centroid information
    prevCentroidInfo = xPredDf[centroidIndex]
    for centroidNumber in rThetaMeasurement['CentroidNumber']:
        measurementCentroidInfo = rThetaMeasurement.loc[centroidNumber]
        length = np.sqrt((prevCentroidInfo.Range**2) + (measurementCentroidInfo.R ** 2) - \
        (2*measurementCentroidInfo.R*prevCentroidInfo.Range*np.cos((prevCentroidInfo.Theta - measurementCentroidInfo.Theta))))
        distanceMatrix[centroidIndex, centroidNumber] = length

#global association
correctAssociationMatrix = np.eye(xPredDf.shape[1],xPredDf.shape[1])
for centroidNumber in xPredDf.columns:
    #find global minimum
    rowIndex, colIndex = (np.where(distanceMatrix == np.min(distanceMatrix))[0][0], np.where(distanceMatrix == np.min(distanceMatrix))[1][0])
    #replace the associated row with inf 
    distanceMatrix[rowIndex, :] = np.Inf
    #row index gives the prev centroid number that associates to the measurement centroid 
    #col index gives the measurement centroid number that associates to the prev centroid 
    correctAssociationMatrix[rowIndex,0] = rThetaMeasurement.loc[colIndex].R #correctly associated range
    correctAssociationMatrix[rowIndex,1] = rThetaMeasurement.loc[colIndex].Theta #correctly associated theta
measurementDf = pd.DataFrame(correctAssociationMatrix, columns=['Range', 'Theta'])
measurementDf['CentroidNumber'] = pd.Series(np.arange(correctAssociationMatrix.shape[0]))
#number of measured centroids is not equal to number of previously known centroids 
#too many centroids
if np.isfinite(distanceMatrix).all():
    #find rows that are not associated
    newCentroids = rThetaMeasurement.values[np.isfinite(distanceMatrix[:,0])]
    measurementDf = pd.concat([measurementDf, pd.DataFrame(newCentroids))
    #if too many
    
    
measurementDf['CentroidNumber'] = pd.Series(np.arange(correctAssociationMatrix.shape[0]))

In [177]:
rThetaMeasurement

,CentroidNumber,R,Theta
0,0,2.692582,-1.190290
1,1,0.860233,0.950547


In [176]:
correctAssociationMatrix

array([[ 0.86023253,  0.95054684],
       [ 2.6925824 , -1.19028995]])

In [124]:
colIndex

1